# Notebook 01 - Exploration de la source OpenF1 API

Ce notebook a pour objectif d’explorer l’API OpenF1 afin de :
- comprendre la structure des données disponibles,
- identifier les champs utiles pour la prédiction du temps au tour,
- valider la couverture historique (2022–2025),
- préparer la définition du schéma de données Bronze.

-----------

## Présentation de l'API OpenF1

OpenF1 est une API publique fournissant des données détaillées de Formule 1,
incluant les sessions, les pilotes et les temps au tour.

Les données sont accessibles via des endpoints REST en format JSON,
sans authentification pour l’usage public.

Documentation : https://openf1.org


-----------

### Imports

In [34]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint


In [33]:
BASE_URL = "https://api.openf1.org/v1"

-----------

## Exploration des meetings (événements)

Les meetings correspondent aux événements de course (Grand Prix).
Ils constituent le niveau le plus haut de structuration des données.

In [ ]:
def get_meetings(year: int):
    url = f"{BASE_URL}/meetings"
    params = {"year": year}
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

meetings_2024 = get_meetings(2024)
pprint(meetings_2024[0])

{'circuit_key': 63,
 'circuit_short_name': 'Sakhir',
 'country_code': 'BRN',
 'country_key': 36,
 'country_name': 'Bahrain',
 'date_start': '2024-02-21T07:00:00+00:00',
 'gmt_offset': '03:00:00',
 'location': 'Sakhir',
 'meeting_code': 'BRN',
 'meeting_key': 1228,
 'meeting_name': 'Pre-Season Testing',
 'meeting_official_name': 'FORMULA 1 ARAMCO PRE-SEASON TESTING 2024',
 'year': 2024}


In [5]:
df_meetings = pd.DataFrame(meetings_2024)
df_meetings.head()

,meeting_key,circuit_key,circuit_short_name,meeting_code,location,country_key,country_code,country_name,meeting_name,meeting_official_name,gmt_offset,date_start,year
0,1228,63,Sakhir,BRN,Sakhir,36,BRN,Bahrain,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,03:00:00,2024-02-21T07:00:00+00:00,2024
1,1229,63,Sakhir,BRN,Sakhir,36,BRN,Bahrain,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,03:00:00,2024-02-29T11:30:00+00:00,2024
2,1230,149,Jeddah,KSA,Jeddah,153,KSA,Saudi Arabia,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,03:00:00,2024-03-07T13:30:00+00:00,2024
3,1231,10,Melbourne,AUS,Melbourne,5,AUS,Australia,Australian Grand Prix,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,11:00:00,2024-03-22T01:30:00+00:00,2024
4,1232,46,Suzuka,JPN,Suzuka,4,JPN,Japan,Japanese Grand Prix,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,09:00:00,2024-04-05T02:30:00+00:00,2024


Les **champs importants** seront :
- meeting_key
- meeting_name
- location
- country_name
- date_start

-----------

## Exploration des sessions

Chaque meeting est composé de plusieurs sessions :
essais libres, qualifications, course.

In [10]:
def get_sessions(meeting_key: int):
    url = f"{BASE_URL}/sessions"
    params = {"meeting_key": meeting_key}
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

sessions = get_sessions(meetings_2024[2]["meeting_key"])
pprint(sessions)

[{'circuit_key': 149,
  'circuit_short_name': 'Jeddah',
  'country_code': 'KSA',
  'country_key': 153,
  'country_name': 'Saudi Arabia',
  'date_end': '2024-03-07T14:30:00+00:00',
  'date_start': '2024-03-07T13:30:00+00:00',
  'gmt_offset': '03:00:00',
  'location': 'Jeddah',
  'meeting_key': 1230,
  'session_key': 9473,
  'session_name': 'Practice 1',
  'session_type': 'Practice',
  'year': 2024},
 {'circuit_key': 149,
  'circuit_short_name': 'Jeddah',
  'country_code': 'KSA',
  'country_key': 153,
  'country_name': 'Saudi Arabia',
  'date_end': '2024-03-07T18:00:00+00:00',
  'date_start': '2024-03-07T17:00:00+00:00',
  'gmt_offset': '03:00:00',
  'location': 'Jeddah',
  'meeting_key': 1230,
  'session_key': 9474,
  'session_name': 'Practice 2',
  'session_type': 'Practice',
  'year': 2024},
 {'circuit_key': 149,
  'circuit_short_name': 'Jeddah',
  'country_code': 'KSA',
  'country_key': 153,
  'country_name': 'Saudi Arabia',
  'date_end': '2024-03-08T14:30:00+00:00',
  'date_start': 

→ *Ici nous avons les informations du Grand Prix de Jeddah (Arabie Saoudite)*

→ *Infos enregistrées dans la variable **sessions***

Les **champs importants** seront :
- session_key
- session_name
- date_start
- date_end

**→ Focus sur les sessions *Race* (les courses).**

-----------

## Exploration des temps au tour

Les temps au tour constituent la variable cible du projet.
Nous explorons ici leur structure et leur granularité.


In [24]:
def get_laps(session_key: int):
    url = f"{BASE_URL}/laps"
    params = {"session_key": session_key}
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

laps = get_laps(sessions[4]["session_key"])
pprint(laps[0])

{'date_start': None,
 'driver_number': 1,
 'duration_sector_1': None,
 'duration_sector_2': 29.707,
 'duration_sector_3': 29.83,
 'i1_speed': 284,
 'i2_speed': 294,
 'is_pit_out_lap': False,
 'lap_duration': None,
 'lap_number': 1,
 'meeting_key': 1230,
 'segments_sector_1': [2048, 2049, 2051, 2049, 2049, 2051, 2049, 2051, 2049],
 'segments_sector_2': [2049, 2051, 2049, 2051, 2049, 2049, 2049],
 'segments_sector_3': [2051, 2049, 2049, 2049, 2049, 2049, 2049, 2049, 0],
 'session_key': 9480,
 'st_speed': 298}


→ *Ici nous avons les informations du premier tour enregistré de la session (**laps[0]**), pour la session Race du GP de Jeddah (**sessions[4]["session_key"]**)*

In [25]:
df_laps = pd.DataFrame(laps)
df_laps.head(10)

,meeting_key,session_key,driver_number,lap_number,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,is_pit_out_lap,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed
0,1230,9480,1,1,None,NaN,29.707,29.830,284.0,294.0,False,NaN,"[2048, 2049, 2051, 2049, 2049, 2051, 2049, 205...","[2049, 2051, 2049, 2051, 2049, 2049, 2049]","[2051, 2049, 2049, 2049, 2049, 2049, 2049, 204...",298.0
1,1230,9480,16,1,None,NaN,29.838,29.664,280.0,293.0,False,NaN,"[2048, 2049, 2049, 2049, 2051, 2049, 2051, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2051, 2049, 2049, 2051, 2051, 2049, 204...",307.0
2,1230,9480,11,1,None,NaN,29.831,29.801,286.0,313.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2051, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",306.0
3,1230,9480,14,1,None,NaN,30.035,30.287,285.0,295.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",299.0
4,1230,9480,81,1,None,NaN,30.007,30.160,287.0,301.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2051, 205...",304.0
5,1230,9480,4,1,None,NaN,30.023,30.261,283.0,300.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2051, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",300.0
6,1230,9480,44,1,None,NaN,30.084,30.266,286.0,306.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",312.0
7,1230,9480,63,1,None,NaN,29.989,30.438,288.0,311.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2051, 2049, 2049, 2049, 2051]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",314.0
8,1230,9480,22,1,None,NaN,30.431,30.889,286.0,301.0,False,NaN,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2051, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",322.0
9,1230,9480,18,1,None,NaN,30.865,30.889,291.0,308.0,False,NaN,"[2048, 2049, 2049, 2051, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",306.0


→ *Les 10 premiers tours enregistrés de la session*

-----------

## Validation du schéma et des types (Bronze)

In [26]:
df_laps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902 entries, 0 to 901
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   meeting_key        902 non-null    int64  
 1   session_key        902 non-null    int64  
 2   driver_number      902 non-null    int64  
 3   lap_number         902 non-null    int64  
 4   date_start         882 non-null    object 
 5   duration_sector_1  877 non-null    float64
 6   duration_sector_2  901 non-null    float64
 7   duration_sector_3  900 non-null    float64
 8   i1_speed           665 non-null    float64
 9   i2_speed           901 non-null    float64
 10  is_pit_out_lap     902 non-null    bool   
 11  lap_duration       880 non-null    float64
 12  segments_sector_1  901 non-null    object 
 13  segments_sector_2  901 non-null    object 
 14  segments_sector_3  901 non-null    object 
 15  st_speed           900 non-null    float64
dtypes: bool(1), float64(7), in

Ici nos colonnes les plus importantes (*meeting_key, session_key, driver_number, lap_number*) ne contiennent aucune valeur manquante, et un type stable (*int64*).

Le schéma est cohérent avec une source de télémétrie F1. Les types sont volontairement conservés tels que fournis par l’API dans une logique Bronze. Les valeurs manquantes correspondent à des cas métier identifiés et seront traitées en Silver.

---------------

## Granularité et clés d’identification

La granularité des données OpenF1 est la suivante :

> 1 ligne = 1 tour d’un pilote dans une session donnée

Clés identifiées :
- meeting_key
- session_key
- driver_number
- lap_number


## Validation de la granularité lap-level

In [29]:
df_laps[
    ["meeting_key", "session_key", "driver_number", "lap_number"]
].head(30)

,meeting_key,session_key,driver_number,lap_number
0,1230,9480,1,1
1,1230,9480,16,1
2,1230,9480,11,1
3,1230,9480,14,1
4,1230,9480,81,1
5,1230,9480,4,1
6,1230,9480,44,1
7,1230,9480,63,1
8,1230,9480,22,1
9,1230,9480,18,1


La granularité lap-level est confirmée par la répétition des identifiants sur plusieurs lignes (*driver_number*). Lorsqu'un pilote complète un tour, son identifiant réapparait et *lap_number* est incrémenté.

## Validation de la clé composite

In [32]:
duplicate_mask = df_laps.duplicated(
    subset=["meeting_key", "session_key", "driver_number", "lap_number"]
)

print("Nombre de doublons détectés :", duplicate_mask.sum())

Nombre de doublons détectés : 0


La combinaison de ces 4 clés est bien unique, stable et robuste. C'est une bonne clé primaire.

------------

## Analyse métier des tours

Un tour de Formule 1 n’est pas toujours représentatif de la performance réelle
d’un pilote ou d’une voiture.

Les tours marqués `is_pit_out_lap = True` correspondent à des tours de sortie
des stands (out-laps). Lors de ces tours :
- le pilote démarre à faible vitesse,
- les pneus et les freins ne sont pas à température optimale,
- l’objectif n’est pas la performance mais la mise en conditions du véhicule.

Ces tours présentent généralement :
- des durées de secteurs manquantes ou partielles,
- une valeur `lap_duration` absente ou non comparable,
- des vitesses intermédiaires plus faibles.

D’un point de vue métier, ces tours sont donc **non représentatifs** de la
performance pure sur un tour lancé. Ils doivent être exclus de l’entraînement
du modèle de prédiction du temps au tour, mais sont conservés dans la couche
Bronze afin de préserver l’information brute et permettre des analyses
exploratoires ultérieures.

À noter : l’API OpenF1 ne fournit pas d’indicateur explicite de `pit-in`.
Les tours d’entrée aux stands devront être identifiés ultérieurement à partir
de signaux indirects (durée anormalement longue, secteurs manquants, rupture de
séquence de tours).


-----

## Analyse de la variable cible

La variable `lap_duration` représente le temps total nécessaire à un pilote
pour compléter un tour de circuit lors d’une session donnée.

D’un point de vue métier, cette variable est centrale :
- elle mesure directement la performance sur un tour,
- elle est continue et numérique,
- elle est comparable entre pilotes dans un même contexte de session.

La cohérence de `lap_duration` est confirmée par sa relation avec les durées de
secteurs :

> `lap_duration ≈ duration_sector_1 + duration_sector_2 + duration_sector_3`

In [36]:
# Conversion des laps en DataFrame
laps_df = pd.DataFrame(laps)

valid_laps = (
    laps_df
    .dropna(
        subset=[
            "lap_duration",
            "duration_sector_1",
            "duration_sector_2",
            "duration_sector_3"
        ]
    )
    .copy()
)

valid_laps["sum_sectors"] = (
    valid_laps["duration_sector_1"]
    + valid_laps["duration_sector_2"]
    + valid_laps["duration_sector_3"]
)

valid_laps["delta"] = (
    valid_laps["lap_duration"] - valid_laps["sum_sectors"]
).abs()


# Statistiques descriptives de l'écart
valid_laps["delta"].describe()

count    8.760000e+02
mean     3.942052e-15
std      6.718808e-15
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.421085e-14
max      2.842171e-14
Name: delta, dtype: float64

Les écarts sont **extrêmement faibles** :
- moyenne ≈ 0
- maximum ≈ 2.84 × 10⁻¹⁴

Cela confirme que `lap_duration ≈ duration_sector_1 + duration_sector_2 + duration_sector_3` pour tous les tours complets.

Les légères différences observées peuvent s’expliquer par :
- des arrondis numériques,
- des données de secteurs manquantes,
- des interruptions de tour (pit-out, incidents).

Les valeurs manquantes de `lap_duration` correspondent majoritairement à des cas métier identifiés (tours incomplets, tours de sortie des stands) et ne traduisent pas une anomalie structurelle de la donnée.

Compte tenu de ces éléments, `lap_duration` constitue une **excellente candidate comme variable cible** pour un modèle de prédiction du temps au tour, à condition de filtrer les tours non représentatifs (*is_pit_out_lap = True ou tours incomplets*) dans les couches de transformation ultérieures.

------------

## Analyse des valeurs manquantes

In [ ]:
df_laps.isna().mean().sort_values(ascending=False)

i1_speed             0.262749
duration_sector_1    0.027716
lap_duration         0.024390
date_start           0.022173
duration_sector_3    0.002217
st_speed             0.002217
i2_speed             0.001109
duration_sector_2    0.001109
segments_sector_2    0.001109
segments_sector_1    0.001109
segments_sector_3    0.001109
lap_number           0.000000
meeting_key          0.000000
session_key          0.000000
driver_number        0.000000
is_pit_out_lap       0.000000
dtype: float64

Les valeurs manquantes sont majoritairement concentrées sur des variables de mesure et correspondent à des situations de course spécifiques. Elles sont conservées en Bronze pour ne pas perdre d’information métier :

- *data_start* : probablement tours incomplets, pit-out ou données non remontées.
- *duration_sector* : un secteur peut ne pas être mesuré si le tour est interrompu ou si le pilote sort des stands (= cas de course).
- *i1_speed* (vitesse intermédiaire à un point précis du circuit) : point de mesure non atteint, données partielles, capteurs limités (= info métier).

-------------------

## Analyse de volumétrie et de cardinalité

In [30]:
print("Nombre total de tours :", len(df_laps))
print("Nombre de meetings :", df_laps["meeting_key"].nunique())
print("Nombre de sessions :", df_laps["session_key"].nunique())
print("Nombre de pilotes :", df_laps["driver_number"].nunique())

Nombre total de tours : 902
Nombre de meetings : 1
Nombre de sessions : 1
Nombre de pilotes : 20


Ici *df_laps* prend bien en compte **1 seul meeting** (le GP de Jeddah), **une seule session** (Race, la course). Il y a bien **20 pilotes** uniques, qui ont réalisé ensemble **902 tours**.

Chaque ligne représente un tour effectué par un pilote lors d’une session donnée. Le nombre total de lignes est cohérent avec le nombre de pilotes et la durée de la session. Ici environ 45 tours par pilote.

Un GP de F1 compte en moyenne 50 tours lors d'une course. Certains pilotes vont terminer les 50, d'autres non pour abandon ou accident, et d'autres ne feront pas le nombre de tour max. Les 45 tours moyens sont donc un résultat cohérent.

--------------

## Limites et décisions pour l’ETL Bronze

Observations :
- Données bien structurées et numériques
- Couverture historique récente mais suffisante (2022–2025) *(ère technique de "l'effet de sol" pour les voitures)*
- Volume de données potentiellement élevé
- Les pit-in ne sont pas indiqués directement. Déduis par tour suivant manquant / lap_duration anormalement long / absence de secteur. A documenter et enrichir.

Décisions :
- Stocker les données OpenF1 en Bronze sans agrégation ou modification
- Filtrer les sessions "Race" dès la couche Silver
- Conserver les pit-out (tour de sortie des stands) pour analyse future

--------------

## Conclusion de l'analyse

L’exploration de l’API OpenF1 montre qu’elle constitue une source de données
solide et adaptée à l’objectif du projet F1PA.

Les données présentent les caractéristiques suivantes :
- une granularité claire et cohérente (un tour par pilote et par session),
- des clés d’identification stables et uniques,
- une structure numérique exploitable pour des modèles de régression,
- des anomalies explicables par le contexte métier de la Formule 1,
- une couverture historique suffisante (2022–2025).

Les données OpenF1 sont donc parfaitement adaptées à une ingestion en couche
Bronze, sans transformation ni agrégation, dans une logique de conservation de
la donnée brute.

Cependant, ces données ne contiennent pas de contexte environnemental
(météo, caractéristiques du circuit), éléments essentiels pour expliquer la
variabilité des temps au tour. Ces dimensions seront apportées par des sources
complémentaires, explorées dans les notebooks suivants (Meteostat et
Wikipedia).

Ce notebook valide ainsi OpenF1 comme **source principale des données de
performance au tour** pour la suite du pipeline ETL et du développement des
modèles de prédiction.